In [ ]:
import pandas as pd
import os

# 현재 디렉토리에서 .csv 파일 모두 탐색
csv_files = [f for f in os.listdir() if f.endswith('.csv')]

# 파일별 컬럼명 출력
for file in csv_files:
    try:
        # 파일 읽기 (인코딩 시도)
        try:
            df = pd.read_csv(file, encoding='utf-8')
        except:
            df = pd.read_csv(file, encoding='cp949')
        
        # 결과 출력
        print(f"📁 파일: {file}")
        print(f"🔹 컬럼명: {list(df.columns)}\n")

    except Exception as e:
        print(f"❌ 오류 발생 - 파일: {file}, 오류: {e}")


In [ ]:
import pandas as pd
import os

# ✅ 분석할 CSV 파일 목록
file_info = [
    ("한국남부발전(주)_성산풍력발전실적_20250228.csv", "년월일", None, "단계", None),
    ("한국남부발전(주)_한경풍력발전실적_20250228.csv", "년월일", None, "단계", None),
    ("한국동서발전(주)_신재생설비 발전량_20230731.csv", "일자", "발전기명", None, "설비용량(MW)"),
    ("한국서부발전(주)_풍력 발전 현황_20230630.csv", "년월일", "발전기명", None, "설비용량(MW)"),
    ("한국서부발전_시간별발전량_풍력.csv", "날짜", "발전기명", None, "설비용량(MW)"),
    ("한국중부발전(주)_풍력 발전 실적(상명풍력).csv", "기준일", None, "호기", None)
]

# ✅ 통합 데이터 저장용 리스트
all_data = []

# ✅ 파일별 처리
for fname, date_col, plant_col, unit_col, cap_col in file_info:
    try:
        # 1) 파일 읽기
        try:
            df = pd.read_csv(fname, encoding='utf-8')
        except:
            df = pd.read_csv(fname, encoding='cp949')
        
        # 2) 날짜 처리
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
        df = df.dropna(subset=[date_col])
        df = df.rename(columns={date_col: "일자"})

        # 3) 발전소명 처리
        if plant_col and plant_col in df.columns:
            df["발전소명"] = df[plant_col].astype(str).str.strip()
        else:
            # 파일명에서 추출 (예: '성산풍력', '한경풍력', '상명풍력' 등)
            name = os.path.basename(fname).split("_")[1].replace("발전실적", "").replace(".csv", "").strip()
            df["발전소명"] = name

        # 4) 호기 처리
        if unit_col and unit_col in df.columns:
            df["호기"] = df[unit_col]
        else:
            df["호기"] = 1

        # 5) 설비용량 처리
        if cap_col and cap_col in df.columns:
            df["설비용량"] = df[cap_col]
        else:
            df["설비용량"] = None

        # 6) 시간별 발전량 처리
        # (시간 컬럼 추출)
        hour_cols = [col for col in df.columns if str(col).strip().replace("시", "").isdigit()]
        for hcol in hour_cols:
            try:
                hour = int(''.join(filter(str.isdigit, str(hcol))))
            except:
                continue
            temp = df[["일자", "발전소명", "호기", "설비용량", hcol]].copy()
            temp = temp.rename(columns={hcol: "발전량"})
            temp["시간"] = hour
            temp["파일명"] = fname
            all_data.append(temp)

        # 7) 예외 케이스: 시간 컬럼이 따로 있는 경우
        if "시간" in df.columns and "해당 시간 발전량(kWh)" in df.columns:
            temp = df[["일자", "발전소명", "호기", "설비용량", "시간", "해당 시간 발전량(kWh)"]].copy()
            temp = temp.rename(columns={"해당 시간 발전량(kWh)": "발전량"})
            temp["파일명"] = fname
            all_data.append(temp)

    except Exception as e:
        print(f"❌ 오류 발생: {fname} → {e}")

# ✅ 통합 DataFrame 생성
df_result = pd.concat(all_data, ignore_index=True)
df_result["시간"] = pd.to_numeric(df_result["시간"], errors="coerce").astype("Int64")

# ✅ 컬럼 정렬 및 출력
df_result = df_result[["파일명", "발전소명", "호기", "일자", "시간", "발전량", "설비용량"]]
print(df_result.head())
